In [ ]:
import os
import scipy.io
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from PIL import Image
from sklearn.model_selection import train_test_split

# --- CONFIGURATION ---
DATA_ROOT = r"D:/deep_learning/stanford-cars-dataset"
OUTPUT_DIR = "processed_data"
BUNDLE_PATH = "resnet50_inference_bundle.pth"
TEST_CSV_PATH = f"{OUTPUT_DIR}/test.csv"

# Create output folder if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Working on device: {device}")

In [ ]:
def process_data_splits():
    print(" Reading Labeled Data...")
    
    # 1. Load the LABELED data (cars_train_annos.mat)
    mat_path = os.path.join(DATA_ROOT, 'car_devkit', 'devkit', 'cars_train_annos.mat')
    
    if not os.path.exists(mat_path):
        print(f" Error: Metadata not found at {mat_path}")
        return

    mat = scipy.io.loadmat(mat_path)
    annotations = mat['annotations'][0]
    
    # 2. Extract Filenames and Labels
    data = []
    for ann in annotations:
        fname = ann[-1][0]
        label = ann[-2][0][0] - 1  # Convert 1-based Matlab index to 0-based Python
        
        # Build full path and check existence
        full_path = os.path.join(DATA_ROOT, 'cars_train', 'cars_train', fname)
        if os.path.exists(full_path):
            data.append({'filepath': full_path, 'label': label})
    
    df = pd.DataFrame(data)
    print(f"Loaded {len(df)} images.")

    # 3. Stratified Splitting (70% Train / 15% Val / 15% Test)
    # First, split off the TEST set (15%)
    train_val_df, test_df = train_test_split(
        df, test_size=0.15, stratify=df['label'], random_state=42
    )
    
    # Next, split the remaining into TRAIN and VALIDATION
    # (0.176 of 85% is roughly 15% of the total)
    train_df, val_df = train_test_split(
        train_val_df, test_size=0.176, stratify=train_val_df['label'], random_state=42
    )

    # 4. Save to CSV
    train_df.to_csv(f"{OUTPUT_DIR}/train_split.csv", index=False)
    val_df.to_csv(f"{OUTPUT_DIR}/val_split.csv", index=False)
    test_df.to_csv(f"{OUTPUT_DIR}/test_split.csv", index=False)

    print("\n Splits Saved to folder 'ready_data_splits':")
    print(f"    Train: {len(train_df)} images (For ResNet/ViT/VGG training)")
    print(f"    Val:   {len(val_df)} images (For Tuning/Early Stopping)")
    print(f"    Test:  {len(test_df)} images (For Final Evaluation Only)")

# Run the function
process_data_splits()

In [ ]:
%%writefile model_utils.py
import torch
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
import pandas as pd

# ==========================================
# 1. STANDARDIZED PREPROCESSING
# ==========================================
def get_transforms(img_size=(224, 224)):
    """
    Returns the standard transforms for all models (ResNet, ViT, VGG).
    """
    # Standard ImageNet statistics
    stats = ((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    
    return {
        # Strong Augmentation for Training
        'train': transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.RandomResizedCrop(img_size, scale=(0.8, 1.0)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(15),
            transforms.ColorJitter(brightness=0.2, contrast=0.2),
            transforms.ToTensor(),
            transforms.Normalize(*stats)
        ]),
        # Clean for Validation/Testing
        'val': transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.CenterCrop(img_size),
            transforms.ToTensor(),
            transforms.Normalize(*stats)
        ]),
        'test': transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.CenterCrop(img_size),
            transforms.ToTensor(),
            transforms.Normalize(*stats)
        ])
    }

# ==========================================
# 2. UNIVERSAL DATASET CLASS
# ==========================================
class UniversalCarDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.df = pd.read_csv(csv_file)
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = row['filepath']
        label = int(row['label'])
        
        try:
            image = Image.open(img_path).convert('RGB')
        except:
            # Fallback for corrupt images (black image)
            image = Image.new('RGB', (224, 224))
            
        if self.transform:
            image = self.transform(image)
            
        return image, label